## Counter Cache With Condition

思路是继续再`post`表里面加字段，记录带条件的聚合查询结果：

```ruby
belongs_to :folder
counter_cache_with_conditions :folder, :messages_count, {}
counter_cache_with_conditions :folder, :unread_messages_count, :unread => true
counter_cache_with_conditions :folder, :unread_messages_count, [:read, :source], lambda{|read, source| read == false && source == 'message'}
counter_cache_with_conditions :folder, :published_events_count, [:published_at], lambda{|published_at| published_at != nil }
```

## 物化视图

```sql
CREATE MATERIALIZED VIEW post_agg_mv AS
SELECT posts.id AS post_id, COUNT(comments.id) AS comments_count, COUNT(comments.apporved = 1)  AS approved_comments_count 
FROM posts 
INNER JOIN comments ON posts.id = comments.post_id 
GROUP BY posts.id
ORDER BY posts.id ASC
```

## Selective Aggregates https://modern-sql.com/feature/filter

https://ruby-china.org/topics/37712

ALL ROWS

```
"a"	1
"a"	2
"a"	3
"b"	3
```

```sql
select count(*) filter(where number > 1),count(*) filter(where number = 1), name  from selectagg group by name
```

```
"2"	"1"	"a"
"1"	"0"	"b"
```